In [396]:
import pandas as pd
import numpy as np
import itertools
import copy
import requests
import datetime
from datetime import date
import operator
import functools
import math
import time
import scipy.stats as stats
from bs4 import BeautifulSoup
from IPython.core.display import display, HTML
from espn_api.basketball import League

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 200)
display(HTML("<style>.container { width:90% !important; }</style>"))

acronym_to_team_dict = {'ATL': 'Atlanta Hawks',
                        'BOS': 'Boston Celtics',
                        'BKN': 'Brooklyn Nets',
                        'CHA': 'Charlotte Hornets',
                        'CHI': 'Chicago Bulls',
                        'CLE': 'Cleveland Cavaliers',
                        'DAL': 'Dallas Mavericks',
                        'DEN': 'Denver Nuggets',
                        'DET': 'Detroit Pistons',
                        'GSW': 'Golden State Warriors',
                        'HOU': 'Houston Rockets',
                        'IND': 'Indiana Pacers',
                        'LAC': 'Los Angeles Clippers',
                        'LAL': 'Los Angeles Lakers',
                        'MEM': 'Memphis Grizzlies',
                        'MIA': 'Miami Heat',
                        'MIL': 'Milwaukee Bucks',
                        'MIN': 'Minnesota Timberwolves',
                        'NOP': 'New Orleans Pelicans',
                        'NYK': 'New York Knicks',
                        'OKC': 'Oklahoma City Thunder',
                        'ORL': 'Orlando Magic',
                        'PHL': 'Philadelphia 76ers',
                        'PHO': 'Phoenix Suns',
                        'POR': 'Portland Trail Blazers',
                        'SAC': 'Sacramento Kings',
                        'SAS': 'San Antonio Spurs',
                        'TOR': 'Toronto Raptors',
                        'UTA': 'Utah Jazz',
                        'WAS': 'Washignton Wizards'
                        }

team_to_acronym_dict = {'Atlanta Hawks': 'ATL',
                        'Boston Celtics': 'BOS',
                        'Brooklyn Nets': 'BKN',
                        'Charlotte Hornets': 'CHA',
                        'Chicago Bulls': 'CHI',
                        'Cleveland Cavaliers': 'CLE',
                        'Dallas Mavericks': 'DAL',
                        'Denver Nuggets': 'DEN',
                        'Detroit Pistons': 'DET',
                        'Golden State Warriors': 'GSW',
                        'Houston Rockets': 'HOU',
                        'Indiana Pacers': 'IND',
                        'Los Angeles Clippers': 'LAC',
                        'Los Angeles Lakers': 'LAL',
                        'Memphis Grizzlies': 'MEM',
                        'Miami Heat': 'MIA',
                        'Milwaukee Bucks': 'MIL',
                        'Minnesota Timberwolves': 'MIN',
                        'New Orleans Pelicans': 'NOP',
                        'New York Knicks': 'NYK',
                        'Oklahoma City Thunder': 'OKC',
                        'Orlando Magic': 'ORL',
                        'Philadelphia 76ers': 'PHL',
                        'Phoenix Suns': 'PHO',
                        'Portland Trail Blazers': 'POR',
                        'Sacramento Kings': 'SAC',
                        'San Antonio Spurs': 'SAS',
                        'Toronto Raptors': 'TOR',
                        'Utah Jazz': 'UTA',
                        'Washington Wizards': 'WAS'
                       }

C:\Users\Jack\AppData\Local\Temp\ipykernel_64092\706589935.py:14: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [427]:
def date_converter(d):
    """Converting date to a string
    Args:
        d: date
    Returns:
        concat_date: string date
    """
    month_day = d.split(',')[1]
    month = month_day.split(' ')[1]
    if month == 'Oct':
        month_num = '10'
    elif month == 'Nov':
        month_num = '11'
    elif month == 'Dec':
        month_num = '12'
    elif month == 'Jan':
        month_num = '01'
    elif month == 'Feb':
        month_num = '02'
    elif month == 'Mar':
        month_num = '03'
    elif month == 'Apr':
        month_num = '04'
    else:
        month_num = '00'
        
    day_num = str(month_day.split(' ')[2]).zfill(2)   # Lpadding with a 0
    year = str(d.split(', ')[2])[-2:]   # Shortening to two numbers
    concat_date = month_num+'/'+day_num+'/'+year
    
    return concat_date

def writing_data(df, filename):
    """Writing to spreadsheet
    Args:
        df: data
        file_name: path of file
    """
    df.to_csv(filename, index=False)
    
def reading_data(file_name):
    """Loads spreadsheet
    Args:
        file_name: path of file
    Returns:
        dataframe of data
    """
    return(pd.read_csv(file_name))
    
def date_to_week(d):
    """Fantasy matchup week. This is based on hard-coded dates. Next step would be to automate this.
    Args:
        d: date
    Returns:
        week_num: fantasy week num
    """
    week_num = 0
    
    if d <= datetime.datetime(2023, 10, 29):
        week_num = 1
    elif d <= datetime.datetime(2023, 11, 5):
        week_num = 2
    elif d <= datetime.datetime(2023, 11, 12):
        week_num = 3
    elif d <= datetime.datetime(2023, 11, 19):
        week_num = 4
    elif d <= datetime.datetime(2023, 11, 26):
        week_num = 5
    elif d <= datetime.datetime(2023, 12, 3):
        week_num = 6
    elif d <= datetime.datetime(2023, 12, 10):
        week_num = 7
    elif d <= datetime.datetime(2023, 12, 17):
        week_num = 8
    elif d <= datetime.datetime(2023, 12, 24):
        week_num = 9
    elif d <= datetime.datetime(2023, 12, 31):
        week_num = 10
    elif d <= datetime.datetime(2024, 1, 7):
        week_num = 11
    elif d <= datetime.datetime(2024, 1, 14):
        week_num = 12
    elif d <= datetime.datetime(2024, 1, 21):
        week_num = 13
    elif d <= datetime.datetime(2024, 1, 28):
        week_num = 14
    elif d <= datetime.datetime(2024, 2, 4):
        week_num = 15
    elif d <= datetime.datetime(2024, 2, 11):
        week_num = 16
    elif d <= datetime.datetime(2024, 3, 3):
        week_num = 17
    elif d <= datetime.datetime(2024, 3, 17):
        week_num = 18
    elif d <= datetime.datetime(2024, 3, 31):
        week_num = 19
    else:
        week_num = 0
        
    return week_num
        
def schedule_maker_23_24():
    """Getting 23-24 schedule from Basketball Reference and writing to a local csv"""
    column_names = ['Date', 'Week', 'Start_Time', 'Visitor', 'Visitor_PTS', 'Home', 'Home_PTS', 'Attendance', 'Arena', 'Notes']
    schedule_df = pd.DataFrame(columns=column_names)
    header_name = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'
    headers = {'User-Agent': header_name}

    url_start = 'https://www.basketball-reference.com/leagues/NBA_2024_games-'
    url_end = '.html'
    url_month_list = ['october', 'november', 'december', 'january', 'february', 'march', 'april']
    url_list = []

    for month in url_month_list:
        url = url_start+month+url_end  # Full url with month in it

        source = requests.get(url, headers=headers)
        soup = BeautifulSoup(source.content, 'html.parser')

        table_container = soup.find('div', attrs={'id': 'div_schedule'})
        table = table_container.find('table', attrs={'id': 'schedule'})
        table_name_container = table.find('thead')

        table_body = table.find('tbody')
        for row in table_body.findAll('tr'):  # Parsing table in basketball reference
            game_date = row.find('th', attrs={'data-stat': 'date_game'}).get_text().strip()
            if game_date == 'Date':   # Basketball Reference has intermediate headers with the column names and we don't want to include these
                continue
            else:  # Formatting date to MM/DD/YY
                game_date = date_converter(game_date)
                
            week_num = date_to_week(datetime.datetime.strptime(game_date, '%m/%d/%y'))
            game_time = row.find('td', attrs={'data-stat': 'game_start_time'}).get_text().strip()
            visitor = row.find('td', attrs={'data-stat': 'visitor_team_name'}).get_text().strip()
            visitor_pts = row.find('td', attrs={'data-stat': 'visitor_pts'}).get_text().strip()
            home = row.find('td', attrs={'data-stat': 'home_team_name'}).get_text().strip()
            home_pts = row.find('td', attrs={'data-stat': 'home_pts'}).get_text().strip()
            attendance = row.find('td', attrs={'data-stat': 'attendance'}).get_text().strip()
            arena = row.find('td', attrs={'data-stat': 'arena_name'}).get_text().strip()
            notes = row.find('td', attrs={'data-stat': 'game_remarks'}).get_text().strip()
            
            # Writing games to a dataframe
            schedule_df = pd.concat([schedule_df, pd.DataFrame([[game_date, week_num, game_time, visitor, visitor_pts, home, home_pts, attendance, arena, notes]], columns=column_names)], axis=0, ignore_index=True)

        time.sleep(2)  
 
    writing_data(schedule_df, filename='C:\\Users\\Jack\\Basketball Project\\NBA_Schedule_23_24.csv')   

def week_day_teams(schedule_df): 
    """Creating nested dict with week, dat and teams playing
    Args:
        schedule_df: dataframe with schedule
    Returns:
        week_day_dict: dict with Week, Day, and Teams playing
    """
    week_day_dict = {}
    for index, row in schedule_df.iterrows():

        if 'Week ' + str(row['Week']) not in week_day_dict.keys():
            week_day_dict['Week ' + str(row['Week'])] = {}

        if row['Date'] not in week_day_dict['Week ' + str(row['Week'])].keys():
            week_day_dict['Week ' + str(row['Week'])][row['Date']] = []

        week_day_dict['Week ' + str(row['Week'])][row['Date']].append(team_to_acronym_dict[row['Visitor']])
        week_day_dict['Week ' + str(row['Week'])][row['Date']].append(team_to_acronym_dict[row['Home']])

    return week_day_dict 

def daily_players(week_num, player_dict, week_day_dict, injury_status_removed='OUT'):
    """Creating dict with a list of players for each day in a provided week
    Args:
        week_num: fantasy week number
        player_dict: dict of teams and players
        week_day_dict: dict of teams playing on a given day
        injury_status_removed: ['OUT', 'DAY_TO_DAY', None]  # Will not consider players based on provided injury status
    Returns:
        day_dict: dict with days and what players are playing
    """
    week = 'Week ' + str(week_num)
    weekly_dict = week_day_dict[week]
    day_dict = {}

    for date, teams in weekly_dict.items():
        for team in teams:
            if team not in player_dict.keys():
                continue

            for player in player_dict[team]:
                if injury_status_removed != None:
                    if (player.injuryStatus == 'OUT') or (injury_status_removed == 'DAY_TO_DAY' and player.injuryStatus in ('OUT', 'DAY_TO_DAY')):
                        continue
                    
                if date not in day_dict.keys():
                    day_dict[date] = []
                    
                day_dict[date].append(player)

    return day_dict

def team_player_dict_creator(roster):
    """Creating dict with players on each Pro Team
    Args:
        roster: roster of fantasy players
    Returns:
        team_player_dict: dict with pro teams and their players
    """
    player_team_dict = {}
    team_player_dict = {}

    for player in roster:
        player_team_dict[player] = player.proTeam

    for player, team in player_team_dict.items():
        if team not in team_player_dict.keys():
            team_player_dict[team] = [player]
        else:
            team_player_dict[team].append(player)
   
    return team_player_dict

def daily_projected_points(week_num, day, team_player_dict, week_day_teams_df, injury_status_removed):
    """Finding projected number of points on given day based on provided roster of players
    Args:
        week_num: fantasy week num
        day: calendar day
        team_player_dict
    Returns:
        total projected fantasy points for given day
        list of standard deviations for player points
        number of games played by players
    """
    position_prime_number_dict = {'PG': 3, 'SG': 5, 'SF': 7, 'PF': 11, 'C': 13, 'G': 17, 'F': 19}   # Prime numbers assoicated with each position. Will be used to find optimal lineup
    position_lineup_spots_dict = {'PG': 1, 'SG': 1, 'SF': 1, 'PF': 1, 'C': 1, 'G': 3, 'F': 3}   # Max number of fantasy positions for each player position
    lineup_size = 10  # Size of starting fantasy lineup
    utility_spots = 3  # Number of Utilitiy spots in which any player can play
    
    if day not in daily_players(week_num, team_player_dict, week_day_teams_df, injury_status_removed).keys():  # If there are no games, return 0
        return 0, 0, 0
    else:
        players = daily_players(week_num, team_player_dict, week_day_teams_df, injury_status_removed)[day]   # Players that play on given day
        total_projected_points = 0    # Total projected points based on optimal lineup
        only_position_projected_points = 0   # Projected points for players that are the only ones at a given position (these players are guaranteed to be in fantasy lineup)
        std_dev_list = []   # List that will hold individual standard deviation for players
        lineup_list = []   # List of players that will be in linuep
        final_std_dev = 0    # Standard deviation for players. Will use formula sqrt(std1^2 + std2^2 ...)

        player_point_dict = {}  # dict of players and their avg. points
        std_dev_dict = {}  # Std_dev for each player
        
        for player in players:
            player_point_dict[player] = (player.avg_points+(player.projected_avg_points if player.avg_points==0 else player.avg_points))/2
            std_dev_dict[player] = 10   # Using 10 for now after analyzing team and seeing it be about 10 regardless of player

        player_point_dict = {k: v for k, v in sorted(player_point_dict.items(), key=lambda item: item[1], reverse=True)}  # Sorting dict based on avg. fantasy points
        players_ordered_list = list(player_point_dict.keys())   # List of player sorted by avg. fantasy points scored

        position_player_dict = {'PG': [], 'SG': [], 'SF': [], 'PF': [], 'C': [], 'G': [], 'F': []}  # Dict with positions and the players that are eligible to play in them
        for position in list(position_lineup_spots_dict.keys()):
            for player in players_ordered_list:   # Creating dict with positions and the players that are eligible to play in them
                if position in player.eligibleSlots:
                    position_player_dict[position].append(player)
                    
        positions_filled_or_cant_fill = []  # Positions that are already filled or cant be filled. We don;t need to worry about these
        only_player_at_pos = []  # List of players that are the only ones eligible for a given position

        for pos, l in position_player_dict.items():  # Iterating through list of players for each position
            adjusted_list = [p for p in l if p not in only_player_at_pos]  # List of players for specific position if they are not the only players at a given position
            if pos in ('G', 'F') and pos not in positions_filled_or_cant_fill and len(adjusted_list) <= position_lineup_spots_dict[pos]:  # Seeing if there are not enough G/F to fill out lineup slots
                positions_filled_or_cant_fill.append(pos)  # If not enough guards or forwards to fill lineup slots, add position to empty list
                for pl in adjusted_list:  # For the players at these positions who are guaranteed to play, updating values in various variables
                    only_position_projected_points+=player_point_dict[pl]  # Adding the player's avg. pts to projected pts
                    std_dev_list.append(std_dev_dict[pl])  # Appending std_dev
                    player_point_dict.pop(pl)   # Removing player from original player dict 
                    only_player_at_pos.append(pl)  # Adding player to only_player_at_pos list
                    players_ordered_list.remove(pl)  # Removing player from ordered players list
                    lineup_list.append(pl)   # Add player to daily lineup list
      
        keep_looping = True
        while keep_looping == True:
            keep_looping = False
            for pos, l in position_player_dict.items():   # Finding positions with no players so we can ignore them
                adjusted_list = [p for p in l if p not in only_player_at_pos]
                if len(adjusted_list) == 0 and pos not in positions_filled_or_cant_fill:
                    positions_filled_or_cant_fill.append(pos)
                
                elif len(adjusted_list) == 1 and adjusted_list[0] not in only_player_at_pos:  # For players who are the only ones at a given position
                    only_position_projected_points+=player_point_dict[adjusted_list[0]]
                    std_dev_list.append(std_dev_dict[adjusted_list[0]])  # Appending std_dev
                    positions_filled_or_cant_fill.append(pos)
                    only_player_at_pos.append(adjusted_list[0])
                    players_ordered_list.remove(adjusted_list[0])
                    lineup_list.append(adjusted_list[0])   # Add player to daily lineup list
                    position_player_dict[pos] = []
                    player_point_dict.pop(adjusted_list[0])
                    keep_looping = True
                    
        if len(players_ordered_list) == 0:   # If all players who are playing have been assigned to a fantasy position, return projected number of points
            return only_position_projected_points, std_dev_list, len(lineup_list)  #np.sqrt(sum(map(lambda i: i * i, std_dev_list)))
                
        prime_number_list = [v for k, v in position_prime_number_dict.items() if k not in positions_filled_or_cant_fill]    # Removing prime numbers associated with empty positions 

        for attempted_player_count in range(min(lineup_size-len(positions_filled_or_cant_fill), len(players_ordered_list)), utility_spots, -1):  # Trying to fill out as many spots as possible
            #print("att player count: "+ str(attempted_player_count))
            for player_combo in itertools.combinations(players_ordered_list, attempted_player_count):  # For all combinations of possible players, ordered by most avg. points
                player_prime_num_dict = {}   # For each player, getting prime numbers associated with their positions

                for player in player_combo[:lineup_size]:
                    player_prime_num_dict[player] = []

                    for pos in list(position_lineup_spots_dict.keys()):
                        if pos in player.eligibleSlots:
                            player_prime_num_dict[player].append(position_prime_number_dict[pos])  # Appending each prime number assoicated with remaining players

                found_combo = False 
                combo_mult_list = []
                for combo in itertools.product(*player_prime_num_dict.values()):  # Combinations of player prime numbers
                    if math.prod(combo) in combo_mult_list:  # if we have already tried combo
                        continue
                    combo_mult_list.append(math.prod(combo))   # Changed from sum to math.prod
                    
                    player_position_score = math.prod(combo)   #functools.reduce(operator.mul, combo)   # Multiplying numbers in the given combination

                    for val in ({np.prod(list(itertools.combinations(prime_number_list,attempted_player_count-utility_spots))[num]) 
                                  for num in range(math.comb(len(prime_number_list), attempted_player_count-utility_spots))}):   # Possible combinations of products of prime numbers of availble position

                        if  player_position_score%val == 0:   # Success if the product of player prime numbers is divisible by the product of the position prime numbers
                            found_combo = True      
                            total_projected_points = 0
                            for player in player_combo:   # After success, finding total projected points
                                total_projected_points += player_point_dict[player]
                                std_dev_list.append(std_dev_dict[player])  # Appending std_dev
                                lineup_list.append(player)   # Add player to daily lineup list

                            return (total_projected_points + only_position_projected_points), std_dev_list, len(lineup_list)  #np.sqrt(sum(map(lambda i: i * i, std_dev_list)))

            print("No combo Failure")

        if len(players_ordered_list) <= utility_spots+1:  # If we have <= players left than available utility positions, remaining players go in those positions
            for player in players_ordered_list:   # After success, finding total projected points
                total_projected_points += player_point_dict[player]
                std_dev_list.append(std_dev_dict[player])  # Appending std_dev
                lineup_list.append(player)   # Add player to daily lineup list
                
            return (total_projected_points + only_position_projected_points), std_dev_list, len(lineup_list)  #np.sqrt(sum(map(lambda i: i * i, std_dev_list)))
   
    print("Here: " + str(day))
    print(players_ordered_list)
    print(total_projected_points + only_position_projected_points)
    return 0, 0, 0   # Shouldn't every get here but just in case

def schedule_matrix(schedule, week_num=0, next_7_days=False):
    """Dataframe of teams and whether they play on given dates
    Args:
        schedule: df of NBA schedule
        week_num: fantasy week number
        next_7_days: bool on whether to look at the next 7 days regardless of where in week we are. If False, will return dates from given week_num
    Returns:
        grid_df: df with teams and whether they play on given dates
    """
    if next_7_days:  # If user wants next 7 days, getting schedule for that
        game_schedule = schedule.loc[(datetime.datetime.today().date() <= pd.to_datetime(schedule['Date'], format='%m/%d/%y').dt.date) 
                                        & (pd.to_datetime(schedule['Date'], format='%m/%d/%y').dt.date < (datetime.datetime.today().date() + datetime.timedelta(days=7)))]
    else:  # Getting schedule for whatever week was inputted
        game_schedule = schedule.loc[schedule['Week']==week_num]
        
    df_index = ['# Games']
    df_index.extend(list(team_to_acronym_dict.keys()))
    df_columns = ['# Games']
    df_columns.extend(np.unique(game_schedule['Date'].values))

    grid_df = pd.DataFrame(index=df_index, columns=df_columns)

    for i, row in game_schedule.iterrows():
        grid_df.loc[row['Visitor'], row['Date']] = team_to_acronym_dict[row['Home']]
        grid_df.loc[row['Home'], row['Date']] = team_to_acronym_dict[row['Visitor']]

    for team in grid_df.index:   # Number of games team plays in timeframe
        grid_df.loc[team, '# Games'] = grid_df.count(axis=1)[team]

    for date in grid_df.columns:   # Number of games per day
        grid_df.loc['# Games', date] = int(grid_df.count()[date]/2)

    grid_df.loc['# Games', '# Games'] = ''
    grid_df.replace(np.nan, '', inplace=True)

    return(grid_df)

def player_matrix(focus_team, schedule, week_num=0, next_7_days=False):
    """Dataframe of players and whether they play on given dates
    Args:
        focus_team: fantasy team we are focusing on
        schedule: df of NBA schedule
        week_num: fantasy week number
        next_7_days: bool on whether to look at the next 7 days regardless of where in week we are. If False, will return dates from given week_num
    Returns:
        grid_df: df with players and whether they play on given dates
    """
    if next_7_days:  # If user wants next 7 days, getting schedule for that
        game_schedule = schedule.loc[(datetime.datetime.today().date() <= pd.to_datetime(schedule['Date'], format='%m/%d/%y').dt.date) 
                                        & (pd.to_datetime(schedule['Date'], format='%m/%d/%y').dt.date < (datetime.datetime.today().date() + datetime.timedelta(days=7)))]
    else: # Getting schedule for whatever week was inputted
        game_schedule = schedule.loc[schedule['Week']==week_num]
       

    team_player_dict = team_player_dict_creator(focus_team.roster)
    player_avg_points_dict = {}
    
    for player in focus_team.roster:   # Adding players and their avg. points to a dict
        #if player.injuryStatus != 'ACTIVE':
        #    pass
       # else:   # Adding player's avg points (or projected points if they have no avg)
        player_avg_points_dict[player.name] = (player.avg_points+(player.avg_points if player.projected_avg_points==0 else player.projected_avg_points))/2
    
    player_avg_points_dict = {k: v for k, v in sorted(player_avg_points_dict.items(), key=lambda item: item[1], reverse=True)}

    df_index = ['# Games']
    df_index.extend(list(player_avg_points_dict.keys()))
    df_columns = ['# Games', 'Status']
    df_columns.extend(np.unique(game_schedule['Date'].values))

    grid_df = pd.DataFrame(index=df_index, columns=df_columns)

    for i, row in game_schedule.iterrows():
        if team_to_acronym_dict[row['Visitor']] in team_player_dict.keys():   # Seeing if my fantasy team has any players on away team
            for p in team_player_dict[team_to_acronym_dict[row['Visitor']]]:   
                grid_df.loc[p.name, 'Status'] = '' if p.injuryStatus=='ACTIVE' else 'DTD' if p.injuryStatus=='DAY_TO_DAY' else 'O' if p.injuryStatus=='OUT' else 'U'  # updating Status column
                grid_df.loc[p.name, row['Date']] = team_to_acronym_dict[row['Home']]
        
        if team_to_acronym_dict[row['Home']] in team_player_dict.keys():   # Seeing if my fantasy team has any players on home team
            for p in team_player_dict[team_to_acronym_dict[row['Home']]]:
                grid_df.loc[p.name, 'Status'] = '' if p.injuryStatus=='ACTIVE' else 'DTD' if p.injuryStatus=='DAY_TO_DAY' else 'O' if p.injuryStatus=='OUT' else 'U'  # updating Status column
                grid_df.loc[p.name, row['Date']] = team_to_acronym_dict[row['Visitor']]

    for pl in grid_df.index:   # Number of games player's team plays in timeframe
        grid_df.loc[pl, '# Games'] = grid_df.count(axis=1)[pl]

    for date in grid_df.columns:   # Number of games per day
        grid_df.loc['# Games', date] = int(grid_df.loc[grid_df['Status']!='O'].count()[date])

    grid_df.loc['# Games', '# Games'] = ''    # Empty sapce where # Games rows and columns meet
    grid_df.loc['# Games', 'Status'] = ''    # Empty sapce where # Games row and Status column meet
    grid_df.replace(np.nan, '', inplace=True)

    return(grid_df)

def players_add_drop(week_num, fantasy_team, scenario="no pickups", drop_point_threshold=30, add_point_threshold=18): 
    """Dict with players that should be added or dropped and potential points gained by doing so
    Args:
        week_num: fantasy week number
        fantasy_team: fantasy team we are looking at
        scenario: either "best case", "worst case", or "no pickups". Will decide expected points
        drop_point_threshold: min amount of points a player on the roster can avg. and not be droppable
        add_point_threshold: min amount of points a free agent can avg. to be addable
    Returns:
        player_value_dict: dict that will hold players on roster, players to add, and projected weekly additional points from adding them
    """

    #print("week " + str(week_num))

    orig_roster = copy.copy(fantasy_team.roster)
    week_day_df = week_day_teams(schedule_df)
    cumul_pts = 0   # Cumulative points for all days analyzed in matchup

    full_std_dev_list = []   # List of std dev
    
    player_value_dict = {}   # dict that will hold players on roster, players to add, and projected weekly additional points from adding them
    
    if scenario == "best case":   # Injury status allowed based on user input
        injury_status_removed = None
    elif scenario == "worst case":
        injury_status_removed = "DAY_TO_DAY"
    else:
        injury_status_removed = 'OUT'

    for day in week_day_df['Week ' + str(week_num)].keys(): 
        #print(day)
        std_dev_list = []   # List for daily std dev
    
        if datetime.datetime.strptime(day, '%m/%d/%y').date() < datetime.datetime.today().date():  # If date is in past, no need to evaluate it
            continue

        original_lineup_points = 0   # Expected fantasy points with original lineup
        original_lineup_games_played = 0   # Expected games_played with original lineup

        for temp_day in week_day_df['Week ' + str(week_num)].keys():  # total weekly points if using current fantasy roster
            if temp_day < day:   
                continue

            proj_pts, proj_std_dev_list, proj_games_played = daily_projected_points(week_num, temp_day, team_player_dict_creator(orig_roster), week_day_df, injury_status_removed)
            original_lineup_points+=proj_pts
            original_lineup_games_played+=proj_games_played
            std_dev_list.extend(proj_std_dev_list)
            
        cumul_pts = max(original_lineup_points, cumul_pts)   # Cumulative pts is the higher of itself and original points
        if len(std_dev_list) > len(full_std_dev_list):   # Cumulative std dev list
            full_std_dev_list = std_dev_list.copy()

        for player in fantasy_team.roster:  # Iterating through eah team on roster to see if there is a better option
            if (player.avg_points+(player.projected_avg_points if player.avg_points==0 else player.avg_points))/2 > drop_point_threshold:  # If player avg's more than threshold, don't consider dropping
                continue
            else:
                if player not in player_value_dict.keys():  # Adding dict to hold free agent addition and projected added points
                    player_value_dict[player] = {}
                    
                for free_agent in free_agents:  # Iterating through fantasy free agents
                    if (free_agent.injuryStatus != 'ACTIVE' or   # If free agent isn't healthy or they don't avg. enough points, don't try to add them
                          (free_agent.avg_points+(free_agent.projected_avg_points if free_agent.avg_points==0 else free_agent.avg_points))/2 < add_point_threshold):
                        continue

                    else:
                        temp_roster = copy.copy(orig_roster)   # copy of original team roster
                        temp_roster.remove(player)    # removing focus roster player
                        temp_roster.append(free_agent)   # adding free agent
                        temp_roster_dict = team_player_dict_creator(temp_roster)
                        new_lineup_points = 0   # Points team will have from adding player
                        new_lineup_games_played = 0   # Games team will play from adding player

                        for temp_day in week_day_df['Week ' + str(week_num)].keys():  # Iterating through remaining days in week to see how many points free agent addition will add
                            if temp_day < day:
                                continue
                                
                            proj_pts, proj_std_dev, proj_games_played = daily_projected_points(week_num, temp_day, temp_roster_dict, week_day_df, injury_status_removed)
                            new_lineup_points+=proj_pts
                            new_lineup_games_played+=proj_games_played

                        total_weekly_additional_points =  new_lineup_points - original_lineup_points    # Additional points from free agent compared to original roster
                        total_weekly_additional_games_played =  new_lineup_games_played - original_lineup_games_played    # Additional games_played from free agent compared to original roster

                        if total_weekly_additional_points > 10:   # If we don;t get at least 10 points from free agent, not worth adding them
                            if free_agent not in player_value_dict[player].keys():   # Adding to nested player/free-agent dict
                                player_value_dict[player][free_agent] = [day, total_weekly_additional_points, total_weekly_additional_games_played]
                            elif total_weekly_additional_points >= player_value_dict[player][free_agent][1]:  # if free agent is already in dict but would get more points from adding later, change date and added points
                                player_value_dict[player][free_agent] = [day, total_weekly_additional_points, total_weekly_additional_games_played]

                            player_value_dict[player] = {k: v for k, v in sorted(player_value_dict[player].items(), key=lambda item: item[1][1], reverse = True)}  # Sorting free agent dict for each player by points free agent will add

    final_player_value_dict = {}
    for k, v in player_value_dict.items():  # Formatting dict to be more readbale
        final_player_value_dict[k] = dict(itertools.islice(v.items(), 5))
    
    if scenario == "best case":   # Determining additional points in best case scenario for team
        fp_extra_points = 0
        total_extra_points = 0
        fp_extra_games = 0
        total_extra_games = 0
        
        for dp, v in final_player_value_dict.items():
            for ap, vl in v.items():
                fp_extra_points = vl[1]  # Extra points from first player     
                fp_extra_games = vl[2]  # Extra games for player
                
                for next_dp, next_v in final_player_value_dict.items():
                    if next_dp == dp:
                        continue

                    for next_ap, next_vl in next_v.items():                 
                        if next_ap == ap:   # Can only add player once
                            continue
                            
                        if fp_extra_points+next_vl[1] > total_extra_points:
                            total_extra_points = fp_extra_points+next_vl[1]
                            total_extra_games = fp_extra_games+next_vl[2]
                                          
        
        cumul_pts+=total_extra_points   # Adding additional points to cumulative total
        if total_extra_games > 0:
            for i in range(total_extra_games):
                full_std_dev_list.append(10)   # Adding another number for std dev list
        elif total_extra_games < 0:
            for i in range(-1*total_extra_games):
                full_std_dev_list.pop()   # Removing numbers from std dev list
        
    return final_player_value_dict, cumul_pts, full_std_dev_list

def matchup_predictor(week_num, focus_team):
    """Predicting score of fantasy matchup
    Args:
        week_num: fantasy week number
        focus_team: fantasy team we are focusing on
    Returns:
        player_value_dict: dict that will hold players on roster, players to add, and projected weekly additional points from adding them
    """
    focus_matchup = focus_team.schedule[week_num-1]   # Matchup for the focus team
    opp_team = None    # Will fill in with opposing team
    focus_team_home = False    # Whether or not focus team is home. Needed to extract current matchup scores

    if focus_team != focus_matchup.home_team:    # Determining whether focus team is home or not
        if focus_team != focus_matchup.away_team:
            opp_team = None
        else:
            opp_team = focus_matchup.home_team
    else:
        opp_team = focus_matchup.away_team
        focus_team_home = True

    focus_curr_score = 0
    opp_curr_score = 0
        
    for matchup in league.scoreboard():   # Getting current scores
        if focus_team in [matchup.home_team, matchup.away_team] and week_num <= date_to_week(datetime.datetime.today()):
            focus_curr_score = matchup.home_final_score if focus_team_home else matchup.away_final_score
            opp_curr_score = matchup.away_final_score if focus_team_home else matchup.home_final_score

    focus_add_drop_dict, focus_additional_pts, focus_std_dev_list = players_add_drop(week_num=week_num, fantasy_team=focus_team)
    focus_std_dev = np.sqrt(sum(map(lambda i: i * i, focus_std_dev_list)))
    opp_add_drop_dict, opp_additional_pts, opp_std_dev_list = players_add_drop(week_num=week_num, fantasy_team=opp_team)
    opp_std_dev = np.sqrt(sum(map(lambda i: i * i, opp_std_dev_list)))
    
    print("Focus Pts: " + str(focus_curr_score) + " + " + str(focus_additional_pts) + "+-" + str(round(focus_std_dev)))
    print("Opp Pts: " + str(opp_curr_score) + " + " + str(opp_additional_pts) + "+-" + str(round(opp_std_dev)))
    
    focus_total_act_pred = focus_curr_score + focus_additional_pts
    opp_total_act_pred = opp_curr_score + opp_additional_pts
    
    z_value = (focus_total_act_pred - opp_total_act_pred)/np.sqrt(focus_std_dev**2 + opp_std_dev**2)   # Can be improved but finding z number
    prob = stats.norm.cdf(z_value)
    print("probability of winning: " + str(round(prob*100)) + "%")


def pretty(d, indent=0):
    """Printing a dict in a more readable way
    Args:
        d: nested dict with players on team, player that should replace them, and added points
        indent: indent level
    """
    for key, value in d.items():
        if not bool(value):  # If nested dict is empty, continue
            continue
        if isinstance(value, dict):  # Printin player that is in the key and recursively printing nested dict
            print(key)
            pretty(value, indent+1)
            
        else:
            print('\t' * indent + str(key) + ": " + str(value)) 

In [432]:
add_drop_dict, orig_pts, std_list = players_add_drop(week_num=7, fantasy_team=league.teams[5], scenario="best case")
pretty(add_drop_dict)

Player(Jalen Williams)
	Player(Bruce Brown): ['12/04/23', 13.330000000000041, 1]
	Player(Grayson Allen): ['12/04/23', 13.160000000000082, 1]
	Player(Dyson Daniels): ['12/04/23', 12.259999999999991, 1]
	Player(Al Horford): ['12/04/23', 11.910000000000082, 1]
	Player(Josh Hart): ['12/05/23', 11.120000000000005, 1]
Player(Jordan Clarkson)
	Player(Bruce Brown): ['12/04/23', 12.31000000000006, 1]
	Player(Grayson Allen): ['12/04/23', 12.1400000000001, 1]
	Player(Dyson Daniels): ['12/04/23', 11.240000000000009, 1]
	Player(Al Horford): ['12/04/23', 10.8900000000001, 1]
	Player(Josh Hart): ['12/05/23', 10.100000000000023, 1]
Player(Kevin Huerter)
	Player(Grayson Allen): ['12/05/23', 23.639999999999986, 1]
	Player(Josh Hart): ['12/05/23', 21.600000000000023, 1]
Player(Mike Conley)
	Player(Bruce Brown): ['12/04/23', 20.410000000000082, 1]
	Player(Grayson Allen): ['12/05/23', 20.24000000000001, 1]
	Player(Dyson Daniels): ['12/04/23', 19.339999999999918, 1]
	Player(Al Horford): ['12/04/23', 18.9900

In [429]:
nba_sched_csv = 'C:\\Users\\Jack\\Basketball Project\\NBA_Schedule_23_24.csv'
schedule_df = reading_data(nba_sched_csv) # Retrieving Spreadsheet
league = League(league_id=1762898804, year=2024, espn_s2 = 'AEAkbmf6ntGyDFU%2BZ5MICm2WvwmwQ7SsLFomcaskbGz8VksoIPAE1D3MYbsTpcyiNBklJETozxCt1D0n3U7yat2RFOSQrW%2FqZdbWbsdxVWuRfOGKCfOfLrdDSYm2VsduCiWS0c5JLZ2zXzCg1UQqzReuFgIK8KLGyLj9%2FNe2IGgEgPqjwcqEp6PSBb6EssdzMJmuQMRgWtWIBE%2BRX1Ii%2FqLJTRTfLOchv6DK2%2FY%2BitmWtiY4Oa2n6gyF0Acup9V34nFNYB8IKmAZLpZqz3QFlTPw', swid='{A98C2FBC-429C-481F-8D11-8EDAE4526062}', debug=False)
free_agents = league.free_agents(size = 250)

In [433]:
matchup_predictor(week_num=6, focus_team=league.teams[5])

Focus Pts: 969.0 + 359.01000000000005+-32
Opp Pts: 960.75 + 329.72999999999996+-32
probability of winning: 80%


In [411]:
player_matrix(league.teams[5], schedule_df, week_num=7, next_7_days=False)

,# Games,Status,12/04/23,12/05/23,12/06/23,12/08/23
# Games,,,2,2,11,11
Shai Gilgeous-Alexander,3,,,,HOU,GSW
Damian Lillard,2,,,NYK,,
Jimmy Butler,3,DTD,,,TOR,CLE
Bam Adebayo,3,O,,,TOR,CLE
Karl-Anthony Towns,3,,,,SAS,MEM
Jerami Grant,3,,,,GSW,DAL
Jordan Clarkson,3,DTD,,,DAL,LAC
Jalen Williams,3,,,,HOU,GSW
Jonas Valanciunas,2,,SAC,,,


In [353]:
schedule_matrix(schedule_df, week_num=5, next_7_days=False)

,# Games,11/20/23,11/21/23,11/22/23,11/24/23,11/25/23,11/26/23
# Games,,8,5,14,10,6,8
Atlanta Hawks,4,,IND,BKN,,WAS,BOS
Boston Celtics,4,CHA,,MIL,ORL,,ATL
Brooklyn Nets,3,,,ATL,,MIA,CHI
Charlotte Hornets,3,BOS,,WAS,,,ORL
Chicago Bulls,4,MIA,,OKC,TOR,,BKN
Cleveland Cavaliers,4,,PHL,MIA,,LAL,TOR
Dallas Mavericks,2,,,LAL,,LAC,
Denver Nuggets,4,DET,,ORL,HOU,,SAS
Detroit Pistons,2,DEN,,,IND,,


In [350]:
add_drop_dict, orig_pts, std_list = players_add_drop(week_num=5, fantasy_team=league.teams[5])
pretty(add_drop_dict)

Player(Jalen Williams)
	Player(Lonnie Walker IV): ['11/25/23', 51.41999999999996]
	Player(Grayson Allen): ['11/24/23', 48.180000000000064]
	Player(Dorian Finney-Smith): ['11/24/23', 44.80000000000007]
	Player(Kyle Lowry): ['11/24/23', 44.299999999999955]
	Player(Goga Bitadze): ['11/24/23', 43.539999999999964]
Player(Kevin Huerter)
	Player(Lonnie Walker IV): ['11/25/23', 51.41999999999996]
	Player(Dorian Finney-Smith): ['11/25/23', 44.80000000000001]
	Player(De'Andre Hunter): ['11/25/23', 41.95999999999998]
	Player(Grayson Allen): ['11/25/23', 24.090000000000032]
	Player(Talen Horton-Tucker): ['11/25/23', 23.639999999999986]
Player(Mike Conley)
	Player(Lonnie Walker IV): ['11/25/23', 27.41999999999996]
	Player(Dorian Finney-Smith): ['11/25/23', 20.80000000000001]
	Player(De'Andre Hunter): ['11/25/23', 17.95999999999998]
Player(Brook Lopez)
	Player(Lonnie Walker IV): ['11/25/23', 25.489999999999952]
	Player(Dorian Finney-Smith): ['11/25/23', 18.870000000000005]
	Player(De'Andre Hunter): 

In [299]:
def players_add_drop(week_num, fantasy_team, drop_point_threshold=30, add_point_threshold=18): 
    """Dict with players that should be added or dropped and potential points gained by doing so
    Args:
        week_num: fantasy week number
        fantasy_team: fantasy team we are looking at
        drop_point_threshold: min amount of points a player on the roster can avg. and not be droppable
        add_point_threshold: min amount of points a free agent can avg. to be addable
    Returns:
        player_value_dict: dict that will hold players on roster, players to add, and projected weekly additional points from adding them
    """

    #print("week " + str(week_num))

    orig_roster = copy.copy(fantasy_team.roster)
    week_day_df = week_day_teams(schedule_df)
    cumul_pts = 0   # Cumulative points for all days analyzed in matchup

    prior_points = 0
    original_points = 0
    full_std_dev_list = []   # List of std dev
    
    player_value_dict = {}   # dict that will hold players on roster, players to add, and projected weekly additional points from adding them

    for day in week_day_df['Week ' + str(week_num)].keys(): 
        #print(day)
        std_dev_list = []   # List for daily std dev
    
        if datetime.datetime.strptime(day, '%m/%d/%y').date() < datetime.datetime.today().date():  # If date is in past, no need to evaluate it
            continue

        original_lineup_points = 0   # Expected fantasy points with original lineup

        for temp_day in week_day_df['Week ' + str(week_num)].keys():  # total weekly points if using current fantasy roster
            if temp_day < day:   
                continue

            proj_pts, proj_std_dev_list = daily_projected_points(week_num, temp_day, team_player_dict_creator(orig_roster), week_day_df)
            original_lineup_points+=proj_pts
            std_dev_list.extend(proj_std_dev_list)
            
            #print(original_lineup_points)
        #print("std: " + str(std_dev_list))
        cumul_pts = max(original_lineup_points, cumul_pts)   # Cumulative pts is the higher of itself and original points
        if len(std_dev_list) > len(full_std_dev_list):   # Cumulative std dev list
            full_std_dev_list = std_dev_list.copy()

        for player in fantasy_team.roster:  # Iterating through eah team on roster to see if there is a better option
            if (player.avg_points+(player.projected_avg_points if player.avg_points==0 else player.avg_points))/2 > drop_point_threshold:  # If player avg's more than threshold, don't consider dropping
                continue
            else:
                if player not in player_value_dict.keys():  # Adding dict to hold free agent addition and projected added points
                    player_value_dict[player] = {}
                    
                for free_agent in free_agents:  # Iterating through fantasy free agents
                    if (free_agent.injuryStatus != 'ACTIVE' or   # If free agent isn't healthy or they don't avg. enough points, don't try to add them
                          (free_agent.avg_points+(free_agent.projected_avg_points if free_agent.avg_points==0 else free_agent.avg_points))/2 < add_point_threshold):
                        continue

                    else:
                        temp_roster = copy.copy(orig_roster)   # copy of original team roster
                        temp_roster.remove(player)    # removing focus roster player
                        temp_roster.append(free_agent)   # adding free agent
                        temp_roster_dict = team_player_dict_creator(temp_roster)
                        new_lineup_points = 0   # Points team will have from adding player

                        for temp_day in week_day_df['Week ' + str(week_num)].keys():  # Iterating through remaining days in week to see how many points free agent addition will add
                            if temp_day < day:
                                continue
                                
                            proj_pts, proj_std_dev = daily_projected_points(week_num, temp_day, temp_roster_dict, week_day_df)
                            new_lineup_points+=proj_pts

                        total_weekly_additional_points =  new_lineup_points - original_lineup_points    # Additional points from free agent compared to original roster

                        if total_weekly_additional_points > 10:   # If we don;t get at least 10 points from free agent, not worth adding them
                            if free_agent not in player_value_dict[player].keys():   # Adding to nested player/free-agent dict
                                player_value_dict[player][free_agent] = [day, total_weekly_additional_points]
                            elif total_weekly_additional_points >= player_value_dict[player][free_agent][1]:  # if free agent is already in dict but would get more points from adding later, change date and added points
                                player_value_dict[player][free_agent] = [day, total_weekly_additional_points]

                            player_value_dict[player] = {k: v for k, v in sorted(player_value_dict[player].items(), key=lambda item: item[1][1], reverse = True)}  # Sorting free agent dict for each player by points free agent will add

    final_player_value_dict = {}
    for k, v in player_value_dict.items():  # Formatting dict to be more readbale
        final_player_value_dict[k] = dict(itertools.islice(v.items(), 5))
        
    return final_player_value_dict, cumul_pts, full_std_dev_list

In [ ]:
matchup_predictor(week_num=5, focus_team=league.teams[5])